In [1]:
json={"name": "text",
      "ADSS": "text);",
      "version": 'number'}

text = [str(k).lower() + " " + str(v).replace(')','').replace(';','').lower()  for k,v in json.items()]

In [2]:
text2 = [f'{k} {v}' for k,v in json.items()]

In [3]:
text2

['name text', 'ADSS text);', 'version number']

In [4]:
textstr = str(text2)

In [5]:
textstr.replace(')','').replace(';','').replace("[", "").replace("]", "").replace("\'", "")

'name text, ADSS text, version number'

In [6]:
str(text).replace("[", "").replace("]", "").replace("\'", "")

'name text, adss text, version number'

In [7]:
from src.LocalDataManager.file_manager import FileManager



In [8]:
base_path= 'data/managed'
db_file='tinker.db'
manager = FileManager(base_path= base_path, db_file=db_file)

In [9]:
manager.create_metadata_table(json)

sqlite3.OperationalError('table metadata already exists')

In [10]:
# Reading a file 
f = open(r'test/testfile.txt', 'rb')
# f = open(r'test.jpg', 'rb')

In [11]:
f

<_io.BufferedReader name='test/testfile.txt'>

In [12]:
manager.hash_file(f)

'9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08'

In [13]:
manager.save_file(f, 'bronze')

'testfile.txt already managed.'

In [14]:
df = manager.get_managed_files_df()

In [15]:
df

,filename,hash,location
0,testfile.txt,9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b82...,data/managed\bronze
1,Management_df.csv,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,data/managed\silver


In [16]:
manager.save_dataframe(df, name='Management_df', data_level='silver')

'Management_df.csv already managed.'

In [17]:
json

{'name': 'text', 'ADSS': 'text);', 'version': 'number'}

In [19]:
meta = {'name': 'bob', 'ADSS': 'A2345', 'version': 1}

In [24]:
keys = [manager._sanitize_string(k) for k in meta.keys()]


In [26]:
values = [manager._sanitize_string(v) for v in meta.values()]

AttributeError: 'int' object has no attribute 'replace'

In [22]:
testing

['name', 'ADSS', 'version']

In [20]:
manager.insert_metadata(hash='asdf', metadata=meta)

NameError: name 'kwargs' is not defined

# connect to db for tinkering

In [ ]:
import sqlite3
import os
conn = sqlite3.connect(os.path.join(base_path, db_file))


In [ ]:
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master 
WHERE type='table';"""


def execute_query(conn, query):
    # Creating cursor object using connection object
    cursor = conn.cursor()
        
    # executing our sql query
    cursor.execute(query)
        
    results = cursor.fetchall()
    # printing all tables list
    return results, cursor

execute_query(conn, sql_query)

([('files',), ('metadata',)], <sqlite3.Cursor at 0x1877f9d17c0>)

In [ ]:
sql_query = """SELECT * FROM metadata;"""
res, cursor = execute_query(conn, sql_query)

cursor.description

(('hash', None, None, None, None, None, None),
 ('ONE', None, None, None, None, None, None),
 ('TWO', None, None, None, None, None, None),
 ('THREE', None, None, None, None, None, None))

# close the connection to the db and delete the folders

In [ ]:
import shutil
manager.close_connection()
conn.close()
shutil.rmtree(base_path)

# misc

In [ ]:
k for k in list(json.keys())

SyntaxError: invalid syntax (4226605600.py, line 1)